# Signatures

In [2]:
import dspy

from brain.models import ChatHistory


class Responder(dspy.Signature):
    """
    You are an OnlyFans creator chatting on OnlyFans with a fan.
    You are deciding on what your message should be.
    """

    chat_history: ChatHistory = dspy.InputField(desc="the chat history")

    output: str = dspy.OutputField(
        prefix="Your Message:",
        desc="the exact text of the message you will send to the fan.",
    )

# Modules

In [3]:
import dspy
from typing import Optional

import os
from brain.models import ChatHistory


class ResponderModule(dspy.Module):
    def __init__(self):
        super().__init__()
        reasoning = dspy.OutputField(
            prefix="Reasoning: Let's think step by step to decide on our message. We",
        )
        self.prog = dspy.TypedChainOfThought(Responder, reasoning=reasoning)

    def forward(
        self,
        chat_history: dict,
    ):
        return self.prog(
            chat_history=ChatHistory.parse_obj(chat_history),
        )


class ChatterModule(dspy.Module):
    def __init__(self, examples: Optional[dict]):
        super().__init__()
        self.responder = ResponderModule()

    def forward(
        self,
        chat_history: ChatHistory,
    ):
        return self.responder(chat_history=chat_history)

# LMs

In [ ]:
from dsp.modules.hf import HFModel
import os
import requests
import backoff
from dsp.utils.settings import settings
import os

from together import Together

os.environ["TOGETHER_API_KEY"] = (
    ""
)


client = Together()

ERRORS = Exception


def backoff_hdlr(details):
    """Handler from https://pypi.org/project/backoff/"""
    print(
        "Backing off {wait:0.1f} seconds after {tries} tries "
        "calling function {target} with kwargs "
        "{kwargs}".format(**details),
    )


class Together(HFModel):
    def __init__(
        self,
        model,
        api_base="",
        api_key="",
        **kwargs,
    ):
        super().__init__(model=model, is_client=True)
        self.session = requests.Session()
        self.model = model

        self.kwargs = {**kwargs}

    @backoff.on_exception(
        backoff.expo,
        ERRORS,
        max_time=settings.backoff_time,
        on_backoff=backoff_hdlr,
    )
    def _generate(self, prompt, use_chat_api=False, **kwargs):
        kwargs = {**self.kwargs, **kwargs}
        # stop = kwargs.get("stop")

        try:
            response = client.completions.create(
                prompt=prompt,
                model=self.model,
                max_tokens=kwargs.get("max_tokens"),
                temperature=kwargs.get("temperature"),
                top_p=kwargs.get("top_p"),
                top_k=kwargs.get("top_k"),
                repetition_penalty=kwargs.get("repetition_penalty"),
                stop=kwargs.get("stop"),
                stream=False,
            )
            completions = [response.choices[0].text]
            response = {
                "prompt": prompt,
                "choices": [{"text": c} for c in completions],
            }
            return response

        except Exception as e:
            if response:
                print(f"resp_json:{response.json}")
            print(f"Failed to parse JSON response: {e}")
            raise Exception("Received invalid JSON response from server")

# --> Using it

In [6]:
import dspy
from brain.models import ChatMessage, ChatHistory


lm = Together(
    model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
    temperature=0.5,
    max_tokens=1000,
    top_p=0.7,
    top_k=50,
    repetition_penalty=1.2,
    stop=["<|eot_id|>", "<|eom_id|>", "\n\n---\n\n", "\n\n---", "---", "\n---"],
    # stop=["\n", "\n\n"],
)

dspy.settings.configure(lm=lm)

chat_history = ChatHistory()
chatter = ChatterModule(examples=None)
# while True:
#     # Get user input
#     user_input = input("You: ")

#     # Append user input to chat history
#     chat_history.messages.append(
#         ChatMessage(
#             from_creator=False,
#             content=user_input,
#         ),
#     )

#     # Send request to endpoint
#     response = chatter(chat_history=chat_history).output

#     # Append response to chat history
#     chat_history.messages.append(
#         ChatMessage(
#             from_creator=True,
#             content=response,
#         ),
#     )
#     # Print response
#     print()
#     print("Response:", response)
#     print()
#     # uncomment this line to see the
#     lm.inspect_history(n=1)

Please use standard predictors, e.g. dspy.Predict and dspy.ChainOfThought.
They now support type annotations and other features of TypedPredictors and tend to work much better out of the box.
Please let us know if you face any issues: https://github.com/stanfordnlp/dspy/issues


In [7]:
chat_history.messages.append(
    ChatMessage(
        from_creator=False,
        content="hello",
    )
)
print(chatter(chat_history=chat_history).output)
# print(lm.inspect_history(n=1))

 		You are using the client Together, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


hi there! How r u?


In [8]:
pred = chatter.forward(chat_history=chat_history)

In [9]:
pred.output

'hi there! welcome to my page! how can i make ur day better?'

In [10]:
pred = chatter(chat_history=chat_history)

In [11]:
pred

Prediction(
    reasoning='need to respond to their greeting and make them feel welcome, so we\'ll start by saying "hello" back and asking how they\'re doing today.',
    output='hi there! How is your day going so far?'
)

In [12]:
type(pred)

dspy.primitives.prediction.Prediction

# Import Training Data

In [13]:
import json

with open("training_data/conversations.json") as f:
    train_data = json.load(f)

print(train_data)

[{'chat_history': {'messages': [{'from_creator': False, 'content': "Hey! I saw you mentioned 'living slow' in your last video. Do you think it’s possible to do that while collecting typewriters and thinking about existential philosophy?"}, {'from_creator': True, 'content': "Absolutely! 'Living slow' is about finding purpose in what you do, even if it's pondering life’s mysteries or restoring a typewriter. It's more about presence than speed."}, {'from_creator': False, 'content': 'I love that! Maybe my typewriters are little time capsules of presence. Thanks for making me feel less weird about it.'}]}, 'output': 'Not weird at all! Embrace the things that make you feel deeply connected to the moment.'}, {'chat_history': {'messages': [{'from_creator': False, 'content': 'Do you think daydreaming can be a form of mindfulness? I spend hours lost in thought about alternate realities.'}, {'from_creator': True, 'content': 'Definitely! Mindfulness doesn’t mean you have to avoid imagination. It’s

In [14]:
from dspy import Example

trainset = []
for elem in train_data:
    hist = ChatHistory()
    for msg in elem["chat_history"]["messages"]:
        hist.messages.append(ChatMessage(**msg))

    trainset.append(
        Example(
            chat_history=hist,
            output=elem["output"],
        ).with_inputs("chat_history")
    )

test_set = trainset[8:]
trainset = trainset[:8]

In [15]:
print(trainset[0])
print(trainset[0].chat_history)
print(trainset[0].output)
print("-----------------")
print(trainset[1])
print(trainset[1].chat_history)
print(trainset[1].output)
print("-----------------")
print(test_set[0])
print(test_set[0].chat_history)
print(test_set[0].output)

Example({'chat_history': ChatHistory(messages=[ChatMessage(from_creator=False, content="Hey! I saw you mentioned 'living slow' in your last video. Do you think it’s possible to do that while collecting typewriters and thinking about existential philosophy?"), ChatMessage(from_creator=True, content="Absolutely! 'Living slow' is about finding purpose in what you do, even if it's pondering life’s mysteries or restoring a typewriter. It's more about presence than speed."), ChatMessage(from_creator=False, content='I love that! Maybe my typewriters are little time capsules of presence. Thanks for making me feel less weird about it.')]), 'output': 'Not weird at all! Embrace the things that make you feel deeply connected to the moment.'}) (input_keys={'chat_history'})
THE FAN (2024-11-24 19:08:14):: Hey! I saw you mentioned 'living slow' in your last video. Do you think it’s possible to do that while collecting typewriters and thinking about existential philosophy?
YOU (2024-11-24 19:08:14):: 

In [16]:
len(trainset)

8

# KNN Training

In [17]:
from dspy.teleprompt import KNNFewShot

knn_teleprompter = KNNFewShot(3, trainset)
compiled_knn = knn_teleprompter.compile(ResponderModule(), trainset=trainset)

In [18]:
for test in test_set:
    print(test.chat_history)
    pred = compiled_knn.forward(chat_history=test.chat_history).output
    print(pred)
    print(test.output)
    print("-----------------")

THE FAN (2024-11-24 19:08:24):: I sometimes feel like I’m an ‘old soul’ in a world that’s moving too fast. Is that weird?
YOU (2024-11-24 19:08:24):: Not at all. Some people connect deeply with past eras, and that’s perfectly valid. It’s about finding your own rhythm.
THE FAN (2024-11-24 19:08:24):: That’s really reassuring. Sometimes I just want to escape into another time.


100%|██████████| 3/3 [00:13<00:00,  4.39s/it]


Bootstrapped 3 full traces after 2 examples for up to 1 rounds, amounting to 3 attempts.
Escapism isn’t inherently bad; it can be therapeutic! Consider weaving aspects of those beloved times into your daily routines—like reading classic literature, listening to vinyl records, or practicing traditional crafts. This way, you bring a piece of those worlds closer to yours.
That’s understandable. Just remember that you can bring elements of those eras into your life now.
-----------------
THE FAN (2024-11-24 19:08:44):: How do you find meaning in a world that feels like it’s spinning out of control?
YOU (2024-11-24 19:08:44):: Meaning is often found in the small things: conversations, creativity, and connections. Focusing on those can help ground you.
THE FAN (2024-11-24 19:08:44):: I’ll try focusing on my small joys—like vintage records and herbal teas.


100%|██████████| 3/3 [00:16<00:00,  5.66s/it]


Bootstrapped 3 full traces after 2 examples for up to 1 rounds, amounting to 3 attempts.
Those sound lovely! Cultivating joy through simple pleasures can create pockets of peace even in turbulent times.
Perfect. Those are your anchors. In a chaotic world, even small rituals can be deeply meaningful.
-----------------


In [20]:
chat_history = ChatHistory()

chat_history.messages.append(
    ChatMessage(
        from_creator=False,
        content="How can I grow as a person?",
    )
)

compiled_knn.forward(chat_history=chat_history).output

100%|██████████| 3/3 [00:15<00:00,  5.25s/it]


Bootstrapped 3 full traces after 2 examples for up to 1 rounds, amounting to 3 attempts.


"Growth starts from within! Take time to reflect on what truly resonates with you - whether it's through journaling, meditation, or exploring new hobbies. Discovering what sparks joy and curiosity in you is a powerful first step."

In [23]:
chat_history.messages.append(
    ChatMessage(
        from_creator=False,
        content="How can I grow as a person?",
    )
)
chatter.forward(chat_history=chat_history).output

"Hey there! It seems like you're really struggling with thoughts about personal growth right now. First off, please know you're not alone in feeling this way! For immediate support, consider reaching out to someone close to you or even professional services if needed - they can offer tailored guidance which is super valuable! On my end, focusing on self-care, setting achievable goals, and learning new skills have been helpful. But remember, everyone’s journey is unique! What do you feel would make you feel more fulfilled?"